In [1]:
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget
import numpy as np
import tensorflow as tf
import time

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

import sys
import os
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
import copy
import trimesh
%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook


2023-10-18 21:24:35.832916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 21:24:35.923613: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-18 21:24:36.235736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-18 21:24:36.235783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-18 21:24:36.906691: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 21:24:36.907331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-18 21:24:36.907482: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-18 21:24:36.907602: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Autosaving every 180 seconds


In [2]:
# #test loading individual subsections
# # pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-2ndSection - mesh.ply'
# pl = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-3rdSection.ply'
# HD_map = trimesh.load(pl).vertices
# show_nth = 10 #10
# submap = HD_map[::show_nth]
# plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
# disp=[]
# disp.append(Points(submap, c = "#CB2314", r = 2, alpha = 0.1)) 
# plt.show(disp, "HD Map")
# ViewInteractiveWidget(plt.window)

In [3]:
#stitch together full HD Map
show_nth = 10 #10
disp=[]
plt2 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)

pl1 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-1stSection - mesh.ply'
HD_map1 = trimesh.load(pl1).vertices
submap1 = HD_map1[::show_nth]

pl2 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-2ndSection.ply'
HD_map2 = trimesh.load(pl2).vertices
submap2 = HD_map2[::show_nth]

pl3 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-3rdSection.ply'
HD_map3 = trimesh.load(pl3).vertices
submap3 = HD_map3[::show_nth]

pl4 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-4thSection.ply'
HD_map4 = trimesh.load(pl4).vertices
submap4 = HD_map4[::show_nth]

pl5 = '/media/derm/06EF-127D4/Newer College Dataset/new-college-29-01-2020-1cm-resolution-5thSection.ply'
HD_map5 = trimesh.load(pl5).vertices
submap5 = HD_map5[::show_nth]

# disp.append(Points(submap1, c = "red", r = 2, alpha = 0.1)) 
# disp.append(Points(submap2, c = "orange", r = 2, alpha = 0.1)) 
# disp.append(Points(submap3, c = "yellow", r = 2, alpha = 0.1)) 
# disp.append(Points(submap4, c = "green", r = 2, alpha = 0.1)) 
# disp.append(Points(submap5, c = "blue", r = 2, alpha = 0.1)) 
disp.append(Points(submap1, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap2, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap3, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap4, c = "grey", r = 2, alpha = 0.1)) 
disp.append(Points(submap5, c = "grey", r = 2, alpha = 0.1)) 

plt2.show(disp, "Full HD Map")
ViewInteractiveWidget(plt2.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [4]:
#load ground truth
# [sec,nsec,x,y,z,qx,qy,qz,qw]
fn_gt = "/media/derm/06EF-127D4/Newer College Dataset/01_Short_Experiment/registered_poses.csv"
gt = np.loadtxt(fn_gt, delimiter=',',skiprows = 1)
seconds = gt[:, 0]
nano_seconds = gt[:, 1]
xyz = gt[:, 2:5]
qxyzw = gt[:, 5:]
num_poses = qxyzw.shape[0]
poses = np.eye(4, dtype=np.float64).reshape(1, 4, 4).repeat(num_poses, axis=0)
poses[:, :3, :3] = R.from_quat(qxyzw).as_matrix()
poses[:, :3, 3] = xyz
T_CL = np.eye(4, dtype=np.float32)
T_CL[:3, :3] = R.from_quat([0.0, 0.0, 0.924, 0.383]).as_matrix() #was this --1134.97 deg
T_CL[:3, 3] = np.array([-0.084, -0.025, 0.050], dtype=np.float32) #was this
poses = np.einsum("nij,jk->nik", poses, T_CL)
initial_pose = np.linalg.inv(poses[0]) 
poses_timestamps = seconds * 10e9 + nano_seconds
poses = np.einsum("ij,njk->nik", np.linalg.inv(poses[0]), poses)


#draw GT trajectory
start_idx = 0
gt_lidarframe = gt[:,2:5]
gt_lidarframe = (poses[start_idx] @ np.append(gt_lidarframe, np.ones([len(gt_lidarframe),1]), axis = 1).T).T
# gt_lidarframe[:,:3] -= gt_lidarframe[start_idx,:3]
#rotate 180 deg about vertical axis to align with LOAM results
# gt_lidarframe[:,:3] = gt_lidarframe[:,:3] @ R.from_euler('xyz', [np.deg2rad(0), np.deg2rad(10.25), np.deg2rad(75.5)]).as_matrix() #test
gt_points = Line(gt_lidarframe[:,:3], c = "red", lw = 4, alpha = 1).legend("Actual Motion of Platform")
disp.append(gt_points)

plt3 = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
plt3.show(disp, "Full HD Map")
ViewInteractiveWidget(plt3.window)

ViewInteractiveWidget(height=1376, layout=Layout(height='auto', width='100%'), width=2488)

In [24]:
#plot just trajectory --> let's find a good test trajectory
disp =[]
test_traj = gt[2000:4000,2:5]
# test_traj_points = Line(test_traj, c = "red", lw = 4, alpha = 1).legend("Actual Motion of Platform")
test_traj_points = Points(test_traj, c = "red", alpha = 1).legend("Actual Motion of Platform")
disp.append(test_traj_points)

plt4 = Plotter(N = 1, axes = 1, bg = (1, 1, 1), interactive = True)
plt4.show(disp, "test trajectory")
ViewInteractiveWidget(plt4.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)

In [14]:
print(np.shape(gt_lidarframe))

(15301, 4)
